<a href="https://colab.research.google.com/github/MoronSlayer/Deep-Learning-Projects/blob/main/BERT_huggingface-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 109695, done.
remote: Counting objects: 100% (299/299), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 109695 (delta 143), reused 208 (delta 93), pack-reused 109396
Receiving objects: 100% (109695/109695), 102.20 MiB | 29.68 MiB/s, done.
Resolving deltas: 100% (81130/81130), done.


In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 38.6 MB/s 
     |████████████████████████████████| 6.6 MB 50.8 MB/s 
     |████████████████████████████████| 120 kB 78.0 MB/s 


In [1]:
!pip show transformers

Name: transformers
Version: 4.22.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache
Location: /usr/local/lib/python3.7/dist-packages
Requires: tqdm, huggingface-hub, requests, importlib-metadata, packaging, regex, tokenizers, pyyaml, filelock, numpy
Required-by: 


In [1]:
from transformers import BertTokenizer
bert_cased_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [2]:
from transformers import BertConfig, BertForPreTraining
config = BertConfig()
model = BertForPreTraining(config)

In [47]:
config


BertConfig {
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.22.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [3]:
from transformers import TextDatasetForNextSentencePrediction
dataset = TextDatasetForNextSentencePrediction(
    tokenizer=bert_cased_tokenizer,
    file_path="/content/sample_data/dataset",
    block_size = 2
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:366: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [7]:
len(dataset)

8

In [7]:
dataset[1]

{'input_ids': tensor([  101,  2019,  6207,  4212,  2104,  1996,  3392,  1012,   102,  1037,
          4248,  2829,  4419, 14523,  2058,  1996, 13971,  3899,  1012,   102]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'next_sentence_label': tensor(1)}

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

example = "Newton invents calculus."

print({x : tokenizer.encode(x, add_special_tokens=False) for x in example.split()})

{'Newton': [8446], 'invents': [1999, 15338, 2015], 'calculus.': [19276, 1012]}


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
example = "An apple falls under the tree."
print({x : tokenizer.encode(x, add_special_tokens=False) for x in example.split()})

{'An': [2019], 'apple': [6207], 'falls': [4212], 'under': [2104], 'the': [1996], 'tree.': [3392, 1012]}


In [23]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
example = "A quick brown fox jumps over the lazy dog."
print({x : tokenizer.encode(x, add_special_tokens=False) for x in example.split()})

{'A': [1037], 'quick': [4248], 'brown': [2829], 'fox': [4419], 'jumps': [14523], 'over': [2058], 'the': [1996], 'lazy': [13971], 'dog.': [3899, 1012]}


In [50]:
for i in range(8):
  print(len(bert_cased_tokenizer.convert_ids_to_tokens(dataset[i]['input_ids'].tolist())))

49
70
70
70
70
49
46
62


In [16]:
dataset[7]['next_sentence_label']

tensor(1)

In [4]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=bert_cased_tokenizer, 
    mlm=True,
    mlm_probability= 0.15
)

In [6]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir= "/path/to/output/dir/for/training/arguments",
    overwrite_output_dir=True,
    num_train_epochs=10000,
    per_gpu_train_batch_size= 16,
    save_steps=100,
    save_total_limit=2,
    prediction_loss_only=True,
    do_eval = True,
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    eval_dataset = dataset, 
)

trainer.train()
trainer.save_model("path/to/your/model")

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8
  Num Epochs = 10000
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization step

Step,Training Loss
50,8.005200
100,4.702000
150,2.786400
200,1.545300
250,0.665400
300,0.235700
350,0.079200
400,0.025500
450,0.013700
500,0.010100


Saving model checkpoint to /path/to/output/dir/for/training/arguments/checkpoint-100
Configuration saved in /path/to/output/dir/for/training/arguments/checkpoint-100/config.json
Model weights saved in /path/to/output/dir/for/training/arguments/checkpoint-100/pytorch_model.bin
Deleting older checkpoint [/path/to/output/dir/for/training/arguments/checkpoint-400] due to args.save_total_limit
Saving model checkpoint to /path/to/output/dir/for/training/arguments/checkpoint-200
Configuration saved in /path/to/output/dir/for/training/arguments/checkpoint-200/config.json
Model weights saved in /path/to/output/dir/for/training/arguments/checkpoint-200/pytorch_model.bin
Deleting older checkpoint [/path/to/output/dir/for/training/arguments/checkpoint-500] due to args.save_total_limit
Saving model checkpoint to /path/to/output/dir/for/training/arguments/checkpoint-300
Configuration saved in /path/to/output/dir/for/training/arguments/checkpoint-300/config.json
Model weights saved in /path/to/output

KeyboardInterrupt: ignored

In [69]:
import torch
x = torch.randn( 8, 70)

x.shape


torch.Size([8, 70])

In [70]:
y = torch.ones([8])

y[2]=0
y[3]=0
y.shape

torch.Size([8])

In [44]:
y

tensor([[1.],
        [1.],
        [0.],
        [0.]])

In [45]:

z= torch.mul(x, y)

In [40]:
z.shape

torch.Size([4, 2])

In [71]:
for i in range(len(y)):
  if y[i] == 0:
    x[i]= x[i]*0 - 100
    
    

In [74]:
x[3]

tensor([-100., -100., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100., -100., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100., -100., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100., -100., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100., -100., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100., -100., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100., -100., -100., -100., -100., -100., -100., -100., -100., -100.])

In [7]:
x.view(-1, 2).shape

torch.Size([8, 2])

In [ ]:
CrossEntropyLoss()